# Preprocess Soil CSVs
This notebook preprocesses per-state/year CSV files stored under `Soil_data/2018` and `Soil_data/2020`.
Run the next code cell to: import libraries, resolve the notebook `root` folder (uses `ipynbname` when available), and define helper functions that add `Year`, `Month`, and `Day` columns to each CSV.
Notes: the date parser uses `pandas.to_datetime(..., errors='coerce', dayfirst=True)` and will coerce unparseable dates to `NaT`.

In [ ]:
import os
import pandas as pd
from pathlib import Path
import warnings
# Ignore pandas UserWarning messages (broader match so suppression works reliably)
warnings.filterwarnings("ignore", category=UserWarning)

# Resolve the notebook directory (use ipynbname if available, otherwise fallback to cwd)
try:
    import ipynbname
    root = Path(ipynbname.path()).parent
except Exception:
    root = Path.cwd()

# Folder containing all your CSV files (relative to this notebook's location)
base_folder_2020 = root / 'Soil_data' / '2020'
base_folder_2018 = root / 'Soil_data' / '2018'

def process_csv(csv_path):
    print(f"\n➡ Processing: {csv_path}")

    df = pd.read_csv(csv_path)

    # Standardize column names
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

    # If all columns exist -> skip file
    if all(col in df.columns for col in ["year", "month", "day"]):
        print("✔ Already has Year/Month/Day → Skipping")
        return

    # Detect date column
    date_cols = [c for c in df.columns if "date" in c]
    if not date_cols:
        print("❌ No date column found → skipping")
        return

    date_col = date_cols[0]
    print(f"   Using date column: {date_col}")

    # Parse date safely — removed deprecated infer_datetime_format to avoid warning
    df["date_parsed"] = pd.to_datetime(
        df[date_col],
        errors="coerce",
        dayfirst=True
    )

    # Create new columns
    df["Year"] = df["date_parsed"].dt.year
    df["Month"] = df["date_parsed"].dt.strftime("%b")  # Jan, Feb, Mar
    df["Day"] = df["date_parsed"].dt.day

    # Drop helper column
    df.drop(columns=["date_parsed"], inplace=True)

    # Save back to SAME file
    df.to_csv(csv_path, index=False)
    print("✔ Updated Year / Month / Day added.")


def scan_and_process_all(folder):
    print("\n🔍 Scanning for CSV files...\n")
    
    for root_dir, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".csv"):
                csv_path = os.path.join(root_dir, file)
                process_csv(csv_path)

    print("\n🎉 ALL FILES PROCESSED SUCCESSFULLY!")

# Run the scanner
scan_and_process_all(base_folder_2018)
scan_and_process_all(base_folder_2020)


In [ ]:
# Merge per-year CSVs into single files
# This cell defines `merge_csv_if_not_exists` and runs it for 2018 and 2020.
# If an output file already exists, the function will skip merging to avoid overwriting.
import pandas as pd
import os
from pathlib import Path

# -------------------------------------------
# Function to merge all CSVs inside a folder
# -------------------------------------------
def merge_csv_if_not_exists(base_folder, output_file):
    base_folder = Path(base_folder)

    # If merged file already exists → skip merging
    if os.path.exists(output_file):
        print(f"✔ {output_file} already exists – skipping merging.")
        return

    print(f"⏳ Merging CSV files from: {base_folder}")

    # Collect all CSV files
    all_files = []
    for root_dir, dirs, files in os.walk(base_folder):
        for file in files:
            if file.endswith(".csv"):
                csv_path = os.path.join(root_dir, file)
                all_files.append(csv_path)

    # Safety check
    if not all_files:
        print(f"❌ No CSV files found in {base_folder}")
        return

    # Merge all CSVs
    df_list = [pd.read_csv(file) for file in all_files]
    df = pd.concat(df_list, ignore_index=True)

    # Save merged file
    df.to_csv(output_file, index=False)
    print(f"✔ Merged and saved: {output_file}")


# -------------------------------------------
# Merge 2018 (Only if file doesn't exist)
# -------------------------------------------
merge_csv_if_not_exists(
    base_folder="Soil_data/2018",
    output_file="merged_soil_data_2018.csv"
)

# -------------------------------------------
# Merge 2020 (Only if file doesn't exist)
# -------------------------------------------
merge_csv_if_not_exists(
    base_folder="Soil_data/2020",
    output_file="merged_soil_data_2020.csv"
)

# Cleaning merged files
The next cell loads the merged per-year CSVs, reports shape and NaN counts, and (by default) drops any row that contains a NaN value.
Warning: this cleaning is aggressive — it overwrites the input files. Adjust the code if you prefer a less destructive approach (for example, drop only rows missing critical columns).

In [ ]:
import pandas as pd

def clean_csv(csv_path):

    print(f"\n==============================")
    print(f"Cleaning File: {csv_path}")
    print(f"==============================")

    # Load CSV
    df = pd.read_csv(csv_path)

    # Shape & NaN count
    print("Shape:", df.shape)
    print("\nNaN count per column:")
    print(df.isna().sum())

    # Drop rows with ANY NaN
    df_cleaned = df.dropna(how="any")

    # Save (overwrite)
    df_cleaned.to_csv(csv_path, index=False)

    print(f"\n✔ Cleaned CSV saved: {csv_path}")
    print(f"✔ Final shape: {df_cleaned.shape}")


# ----------------------------
# Clean Both Files
# ----------------------------

clean_csv("merged_soil_data_2018.csv")
clean_csv("merged_soil_data_2020.csv")

print("\n🎉 All files cleaned successfully!")

# Create final combined file
This cell concatenates the cleaned 2018 and 2020 CSVs into a single `merged_final.csv`. It prints a few duplicate rows before removing duplicates and can optionally drop rows with any NaN values.
Review the printed shapes and duplicate samples before changing cleaning flags.

In [ ]:
"""
Merge two soil CSVs into a single merged_final.csv

Added:
- Print first 5 duplicate rows before dropping duplicates
"""

from pathlib import Path
import pandas as pd

# -------- CONFIG --------
# Resolve the notebook directory (use ipynbname if available, otherwise fallback to cwd)
try:
    import ipynbname
    root = Path(ipynbname.path()).parent
except Exception:
    root = Path.cwd()

file_2018 = root / "merged_soil_data_2018.csv"
file_2020 = root / "merged_soil_data_2020.csv"
out_file = root / "merged_final.csv"

drop_any_nan = True       # set True to drop rows that contain any NaN
drop_duplicates = True     # set True to remove exact duplicate rows

# -------- helper --------
def read_csv_fallback(path: Path) -> pd.DataFrame:
    """Try utf-8 then latin1; set low_memory False and skip bad lines."""
    try:
        return pd.read_csv(path, encoding="utf-8", low_memory=False, on_bad_lines="skip")
    except Exception:
        return pd.read_csv(path, encoding="latin1", low_memory=False, on_bad_lines="skip")

# -------- read --------
print("Reading files...")
df18 = read_csv_fallback(file_2018)
print(f"  {file_2018.name}: shape {df18.shape}")

df20 = read_csv_fallback(file_2020)
print(f"  {file_2020.name}: shape {df20.shape}")

# -------- merge --------
print("Concatenating...")
merged = pd.concat([df18, df20], ignore_index=True, sort=False)
print("  merged shape (before cleaning):", merged.shape)

# -------- show 5 duplicates BEFORE cleaning --------
duplicate_rows = merged[merged.duplicated(keep=False)]

print("\n--- First 5 Duplicate Rows (Before Removing Duplicates) ---")
if duplicate_rows.empty:
    print("No duplicates found.")
else:
    print(duplicate_rows.head(5))

# -------- optional cleaning --------
if drop_any_nan:
    before = len(merged)
    merged = merged.dropna(how="any")
    print(f"  dropped {before - len(merged)} rows with any NaN -> shape {merged.shape}")

if drop_duplicates:
    before = len(merged)
    merged = merged.drop_duplicates().reset_index(drop=True)
    print(f"  dropped {before - len(merged)} duplicate rows -> shape {merged.shape}")

# -------- save --------
merged.to_csv(out_file, index=False)
print(f"\n✔ Merged CSV saved to: {out_file}")
print("Final shape:", merged.shape)

# State → Season mapping
The following cell defines `state_season_map`, a dictionary mapping state names to lists of month abbreviations for each season. This is used to derive a `Season` column for each row in the final merged CSV.

In [ ]:
state_season_map = {

    # ====== SOUTH INDIA ======
    "ANDHRA PRADESH": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "TELANGANA": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "TAMILNADU": {
        "Winter": ["JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV", "DEC"]
    },
    "KARNATAKA": {
        "Winter": ["JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV", "DEC"]
    },
    "KERALA": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "LAKSHDWEEP": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "PONDICHERRY": {
        "Winter": ["JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV", "DEC"]
    },

    # ====== NORTH INDIA ======
    "DELHI": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "PUNJAB": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "HARYANA": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "HIMACHAL PRADESH": {
        "Winter": ["NOV", "DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT"]
    },
    "UTTARAKHAND": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "UTTAR PRADESH": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "JAMMU & KASHMIR": {
        "Winter": ["NOV", "DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT"]
    },
    "LADAKH": {
        "Winter": ["OCT", "NOV", "DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG"],
        "Post-monsoon": ["SEP"]
    },
    "CHANDIGARH": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },

    # ====== EAST INDIA ======
    "WEST BENGAL": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "BIHAR": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "JHARKHAND": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "ODISHA": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },

    # ====== WEST INDIA ======
    "GUJARAT": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "RAJASTHAN": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "MAHARASHTRA": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "GOA": {
        "Winter": ["DEC", "JAN"],
        "Summer": ["FEB", "MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "DADRA AND NAGAR HAV": {
        "Winter": ["DEC", "JAN"],
        "Summer": ["FEB", "MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "DAMAN & DIU": {
        "Winter": ["DEC", "JAN"],
        "Summer": ["FEB", "MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },

    # ====== CENTRAL INDIA ======
    "MADHYA PRADESH": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "CHHATTISGARH": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },

    # ====== NORTHEAST INDIA ======
    "ASSAM": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP", "OCT"],
        "Post-monsoon": ["NOV"]
    },
    "ARUNACHAL PRADESH": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "MANIPUR": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP", "OCT"],
        "Post-monsoon": ["NOV"]
    },
    "MEGHALAYA": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP", "OCT"],
        "Post-monsoon": ["NOV"]
    },
    "MIZORAM": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP", "OCT"],
        "Post-monsoon": ["NOV"]
    },
    "NAGALAND": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP", "OCT"],
        "Post-monsoon": ["NOV"]
    },
    "SIKKIM": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "TRIPURA": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP", "OCT"],
        "Post-monsoon": ["NOV"]
    },

    # ====== ISLANDS ======
    "ANDAMAN & NICOBAR": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP", "OCT"],
        "Post-monsoon": ["NOV"]
    },

    None: {
        "Winter": [],
        "Summer": [],
        "Monsoon": [],
        "Post-monsoon": []
    }
}

# Apply season mapping and write results
This cell defines `get_season`, detects the state and month columns in `merged_final.csv`, normalizes month values, computes the `Season` column, prints a summary of unknown mappings, and overwrites the final CSV with the new `Season` column.

In [ ]:
# ---------------------
# get_season function (your original, unchanged)
# ---------------------
def get_season(state, month):
    """
    Returns the season (Winter, Summer, Monsoon, Post-monsoon) 
    based on state and month abbreviation.
    """
    if pd.isna(state) or pd.isna(month):
        return "Unknown"

    state = state.upper().strip()
    month = month.upper().strip()

    if state not in state_season_map:
        return "Unknown"

    season_info = state_season_map[state]

    for season_name, month_list in season_info.items():
        if month in month_list:
            return season_name

    return "Unknown"

# ---------------------
# Read final merged CSV
# ---------------------
df = pd.read_csv(out_file, low_memory=False)
print(f"Loaded {out_file} -> shape {df.shape}")

# ---------------------
# Detect state column robustly
# ---------------------
col_map = {c.lower().strip(): c for c in df.columns}
state_candidates = ["state_name", "state name", "state"]
state_col = None
for cand in state_candidates:
    if cand in col_map:
        state_col = col_map[cand]
        break
if state_col is None:
    raise KeyError(f"State column not found. Available columns: {list(df.columns)}")

# ---------------------
# Normalize Month column and apply Season mapping
# ---------------------
# Ensure Month column exists; common names: 'Month', 'month'
month_col = None
for cand in ["month", "Month"]:
    if cand in df.columns:
        month_col = cand
        break
if month_col is None:
    # create Month col if missing
    df["Month"] = pd.NA
    month_col = "Month"

# Normalize Month values (strip, uppercase). Keep empty as NA
df[month_col] = df[month_col].astype(str).str.strip().replace({"": pd.NA, "nan": pd.NA})
df[month_col] = df[month_col].where(df[month_col].notna(), pd.NA)
df[month_col] = df[month_col].astype(str).str.upper().str.strip().replace({"NAN": pd.NA})
df[month_col] = df[month_col].where(df[month_col].notna(), pd.NA)

# Apply Season using the detected state column
df["Season"] = df.apply(lambda row: get_season(row[state_col], row[month_col]), axis=1)

# ---------------------
# Summary & sample of Unknown
# ---------------------
print("\nSeason value counts:")
print(df["Season"].value_counts(dropna=False))

unknown_count = (df["Season"] == "Unknown").sum()
print(f"\nRows with Season == 'Unknown': {unknown_count}")

if unknown_count > 0:
    print("\nSample Unknown rows (first 10):")
    print(df[df["Season"] == "Unknown"].head(10)[[state_col, month_col, "Season"]])

# ---------------------
# Write back (overwrite) the same merged CSV
# ---------------------
df.to_csv(out_file, index=False)
print(f"\n✔ Updated CSV overwritten at: {out_file} (Season column added/updated)")